# Spatially Informed Traveling Salesman Problem
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Rw9_fomI2CRgYNwEmZRx8XU3j1RQK9gO)
Developed by Wanhee Kim(Phd student at UTK) / Revised Date : 04/09/2024


## Sequence of spatially informed TSP
1. Measure adjacency based on spatial information
2. Generate distance matrix
3. Analyze TSP

## Analysis

In [1]:
pip install spatialtsp

Note: you may need to restart the kernel to use updated packages.


In [2]:
#change path
import os
os.chdir('D:/GIS program/Github/spatialtsp/spatialtsp-5')

In [3]:
# 현재 작업 디렉토리를 출력
current_directory = os.getcwd()
print("현재 작업 디렉토리:", current_directory)


현재 작업 디렉토리: D:\GIS program\Github\spatialtsp\spatialtsp-5


In [4]:
# Make Toy Data
import sys
sys.path.insert(0, 'D:/GIS program/Github/spatialtsp/spatialtsp-5')
from spatialtsp import is_far_enough, generate_clustered_points
points=generate_clustered_points(10)
points

,geometry
0,POINT (45.00000 33.00000)
1,POINT (32.00000 38.00000)
2,POINT (39.00000 33.00000)
3,POINT (5.00000 20.00000)
4,POINT (29.00000 32.00000)
5,POINT (33.00000 37.00000)
6,POINT (10.00000 9.00000)
7,POINT (9.00000 17.00000)
8,POINT (8.00000 15.00000)
9,POINT (20.00000 21.00000)


In [5]:
# Calculate Standard Distance Matrix
import sys
sys.path.insert(0, 'D:/GIS program/Github/spatialtsp/spatialtsp-5')
from spatialtsp import calculate_distance_matrix
distance_matrix=calculate_distance_matrix(points)
distance_matrix

array([[   0, 1393,  600, 4206, 1603, 1265, 4244, 3940, 4115, 2773],
       [1393,    0,  860, 3245,  671,  141, 3640, 3114, 3324, 2081],
       [ 600,  860,    0, 3640, 1005,  721, 3764, 3400, 3585, 2247],
       [4206, 3245, 3640,    0, 2683, 3276, 1208,  500,  583, 1503],
       [1603,  671, 1005, 2683,    0,  640, 2983, 2500, 2702, 1421],
       [1265,  141,  721, 3276,  640,    0, 3624, 3124, 3330, 2062],
       [4244, 3640, 3764, 1208, 2983, 3624,    0,  806,  632, 1562],
       [3940, 3114, 3400,  500, 2500, 3124,  806,    0,  224, 1170],
       [4115, 3324, 3585,  583, 2702, 3330,  632,  224,    0, 1342],
       [2773, 2081, 2247, 1503, 1421, 2062, 1562, 1170, 1342,    0]])

In [ ]:
import spatialtsp
print(dir(spatialtsp))

In [ ]:
import spatialtsp
help(spatialtsp)

In [8]:
# Calculate Distance Matrix with the concept of adjacency
# os.chdir('D:/GIS program/Github/spatialtsp/spatialtsp-5')
# from spatialtsp import voronoi_adjacency_distance, knn_adjacency_distance, combine_distance_matrices
# Voronoi_distance = voronoi_adjacency_distance(points)
# KNN_distance = knn_adjacency_distance(points)
# combined_distance = combine_distance_matrices([Voronoi_distance, KNN_distance])

In [ ]:
# set working directory
# import os

# path = 'D:/GIS_analyzing/1.Standard_TSP/0.test_iteration' # write your own directory
# os.chdir(path)
# os.getcwd()

In [ ]:
## other codes will be updated soon

## Display Basemap

In [ ]:
## Add basemap
from spatialtsp import Map

my_map = Map(center=[40.7128, -74.0060], zoom=10)
my_map.add_basemap("OpenStreetMap.Mapnik")

my_map


In [ ]:
## Add GeoJSON
from spatialtsp import Map

geojson_map = Map(center=[37.0902, -95.7129], zoom=4)
geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_states.geojson"
geojson_map.add_geojson(geojson_url, name="US Counties")

geojson_map


In [ ]:
## Add shp
import geopandas as gpd
import requests
import zipfile
import io
from spatialtsp import Map

# Shapefile URL
url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Download and extract the zip file
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))

# Find the shapefile name
shp_name = [name for name in z.namelist() if '.shp' in name][0]

# Read and extract the shapefile
z.extractall("temp_shp")
states_gdf = gpd.read_file("temp_shp/us_states.shp")


# Display the shapefile
my_map = Map(center=[37.0902, -95.7129], zoom=4)
my_map.add_vector(states_gdf, name="US States")

my_map


In [ ]:
## Add Vector
from spatialtsp import Map

# Initialize the map
my_map = Map(center=[37.0902, -95.7129], zoom=4)

# GeoJSON URLs
city_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_cities.geojson"
county_geojson_url = "https://github.com/opengeos/datasets/releases/download/us/us_counties.geojson"

# Shapefile URL
states_shape_url = 'https://github.com/opengeos/datasets/releases/download/us/us_states.zip'

# Load GeoJSONs as GeoDataFrames
gdf_cities = gpd.read_file(city_geojson_url)
gdf_counties = gpd.read_file(county_geojson_url)

# Use add_vector to add GeoDataFrames to the map
my_map.add_vector(gdf_counties, name="US Counties")
my_map.add_vector(gdf_cities, name="US Cities")
# For the shapefile, directly use the URL
my_map.add_vector(states_shape_url, name="US States")

# Display the map
my_map

##Raster Map

In [ ]:
#change path
import os
os.chdir('D:/GIS program/Github/spatialtsp/spatialtsp-5')

In [ ]:
# import functions
!pip install localtileserver
import spatialtsp
from spatialtsp import Map

In [ ]:
# check the path of functions
from importlib import reload
reload(spatialtsp)

from spatialtsp import Map
m = Map()
print(dir(m)) 

import spatialtsp
print(spatialtsp.__file__)

In [ ]:
## overlay climate image(png)
from ipyleaflet import Map, ImageOverlay

m = Map(center=(25, -115), zoom=4)

image = ImageOverlay(
    url="https://i.imgur.com/06Q1fSz.png",
    # url='../06Q1fSz.png',
    bounds=((13, -130), (32, -100)),
)

m.add(image)
m

In [ ]:
## Overlay GIF of TSP into basemap
m = Map(center=(40, -100), zoom=4)

url = "https://graphdeeplearning.github.io/project/combinatorial-optimization/tsp-gif.gif"

# url='../06Q1fSz.png',
bounds = ((25, -125), (51, -66))
m.add_image(url, bounds)
m.add_layers_control()
# m.scroll_wheel_zoom = True
m

In [ ]:
## Overlay COloud Optimized GeoTiff into Basemap
from localtileserver import TileClient, get_leaflet_tile_layer, examples
from spatialtsp import Map

client = TileClient(
    "https://github.com/opengeos/datasets/releases/download/raster/srtm90.tif"
)

# Create ipyleaflet TileLayer from that server
t = get_leaflet_tile_layer(client)
# Create ipyleaflet map, add tile layer, and display
m = Map()
m.add(t)
m.center = client.center()
m.zoom = client.default_zoom
m